In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import sys
sys.path.append('../segmention_buildings/BuildFormer')
sys.path.append('/home/wanghaifeng/whf_work/work_sync/vit/vit-pytorch/vit_pytorch')
sys.path.append('/home/wanghaifeng/whf_work/work_sync/satellite_data/models_config/segment-anything/segment_anything')

import matplotlib.pyplot as plt
import json
import sys
from torch  import nn
from tqdm import tqdm
from torch import optim
import torch.nn.functional as F
import cv2
import numpy as np
# import kornia as K
import math
from glob import glob
from os.path import join
import torch


In [16]:
from init import *
from train_model import train_model
from lossers import dice_loss
from  models import get_model
from lossers import get_loss_function
from vdatasets import get_dataloader
from optimizers_schedulers import get_optimizer_scheduler, update_paramter
from utiles.mask_utles import get_color_mask,mask2color,show_image_mask,show_image_mask_, rgbmask2label,label_colormap, image_diff
from utiles.mask_utles import rgbmask2label,label2rgbmask,rgb2cls_masks,label2rgbmask2,label_colormap
from utiles.tensor2imgmask import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
##1. 数据配置
# config_path = 'model_config/model_base_unet_seg_dataset_test.json'
config_path = 'model_config/model_vgg_unet_seg_dataset_harvey.json'
cfg = json.load(open(config_path))
# cfg['model']='vit_model'
# cfg['data']['dataset']='mapdataset'
# cfg['data']['use_cls']=['buildings','damaged_buildings']
# cfg['data']['path']=r'/home/wanghaifeng/project_work/datasets/data_workspaces/labeled_data/'
# cfg['data']['n_classes']=3
cfg['training']['batch_size']=1
# cfg['training']['img_size']=(1024,1024)

n_classes=cfg['data']['n_classes']
load_weights =None # './model_works/vgg_unet/vgg_unet_2024_01_23_18_28_53/model.pt'
model_not_dataparal = False

if load_weights and not Path(load_weights).exists():
        load_weights=''

model = get_model(cfg['model'])(3, n_classes)

if model_not_dataparal:
    model.load_state_dict(torch.load(load_weights, map_location='cpu'))
    load_weights = ''
    
model = torch.nn.parallel.DataParallel(model)
if load_weights and len(load_weights)>0:
    print('loading weights',load_weights)
    model.load_state_dict(torch.load(load_weights, map_location='cpu'))

optimizer, scheduler = get_optimizer_scheduler(model,cfg)

get_train_loss_func = get_loss_function(cfg["training"]["loss"].copy())

trainloader, valloader = get_dataloader(cfg)


init model : vgg_unet
init optimizer : rmsprop params:  {'lr': 0.001, 'weight_decay': 1e-05, 'momentum': 0.9}
init scheduler:  steplr params:  {'step_size': 1, 'gamma': 0.7}
init loss : loss_cross_entropy_mutliclas  params: {'mask_key': ['mask1']}
init dataset:  seg_dataset
----------------------- /home/wanghaifeng/project_work/datasets/disaster_dataset/harvey
data config:  {'seed': 1337, 'model': 'vgg_unet', 'data': {'dataset': 'seg_dataset', 'n_classes': 3, 'path': '/home/wanghaifeng/project_work/datasets/disaster_dataset/harvey', 'split': '', 'data_img_size': [1024, 1024], 'use_sub': ['t1', 'mask1'], 'index': 't1', 'limt_num': None, 'used_datainfo': False}, 'training': {'batch_size': 1, 'n_workers': 16, 'img_size': [640, 640], 'optimizer': {'name': 'rmsprop', 'lr': 0.001, 'weight_decay': 1e-05, 'momentum': 0.9}, 'loss': {'name': 'loss_cross_entropy_mutliclas', 'mask_key': ['mask1']}, 'lr_schedule': {'step_size': 1, 'gamma': 0.7}}}
select index: t1  index_n: 1
----------------init da

In [22]:
# model.float().train()
# mask_key=['t1_b','mask1_b1','mask1_2','cd_cd_1','cd_mask_1']
import torchvision.transforms.functional as transF
mask_key = cfg['training']['loss']['mask_key']
for i,data in enumerate(trainloader):   
   
    img = data[list(data.keys())[0]]
    # image_t1   = transF.normalize(img.cuda().float(),[0.31815762,0.32456695,0.29096074],[0.18410079,0.17732723,0.18069517])
    image_t1   = torch.nn.functional.normalize(img.cuda().float())
    # image_mask1_2 = data[mask_key[2]].cuda()[:,0:1]
    # # t1_b   =torch.nn.functional.normalize(data['t1_b'].cuda().float())
    # t1_b = data[mask_key[0]].cuda().float()
    # # mask1_2   =  data[mask_key[2]].cuda()
    # mask_b   =  data[mask_key[1]].cuda().float()/255.0
    # mask_cd   =  data[mask_key[3]].cuda().float()/255.0
    # mask_cd_mask =  data[mask_key[4]].cuda().float()/255.0
    # rs = model(image_t1)
    
#     if i>3:
#         print(i)
#         break
    break


In [25]:
data['t1'].shape

torch.Size([1, 3, 640, 640])

In [14]:
###test model

# img =  cv2.imread("/home/wanghaifeng/project_work/datasets/result_sam/E243100N3303400UTM15R/sum.png")
# img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
# sub_img  = img[320:640,320:640]

# img = cv2.resize(sub_img,(1024,1024))
# img = torch.from_numpy(img.transpose(2, 0, 1))
# image_t1   = torch.nn.functional.normalize(img.unsqueeze(0).cuda().float())

# with torch.no_grad():
#     rs = model(image_t1)
# img =data['t1'].cpu().numpy()
# image_mask1_2 = data[mask_key[0]][:,0]

# rs0 = torch.argmax(rs,1)
# plt.imshow(rs0[0].detach().cpu().numpy()>0)

In [11]:
mask_key=cfg["training"]["loss"]["mask_key"]
loss = get_train_loss_func(rs,data,mask_key=mask_key)

In [4]:
from thop import profile
import torchprof
from fvcore.nn import FlopCountAnalysis, parameter_count_table,flop_count_str,flop_count_table,ActivationCountAnalysis

def params_count(model):
    return np.sum([p.numel() for p in model.parameters()]).item()

def analysis_model(model,x=None):
    # fa = FlopCountAnalysis(model, imgs)
    # print(flop_count_str(FlopCountAnalysis(model, (imgs,imgs))))
    tabel = flop_count_table(FlopCountAnalysis(model, x))
    # acts = ActivationCountAnalysis(model, imgs)
    # acts.by_module()
    return tabel

def analysis_model1(model,x=None):
    flops, params = profile(model, (imgs,))
    with torchprof.Profile(model, use_cuda=False) as prof:
        model(imgs)
    trace, event_lists_dict  = prof.raw()
    return trace, event_lists_dict

In [1]:
# print(analysis_model(model,x=torch.ones(5,3,1024,1024).cuda()))

In [5]:
# import torchvision.models
# import torch
# import torchsummary
# device = torch.device('cpu')
# model.to(device)
# torchsummary.summary(model.cuda(), (3, 224, 224))